# Lab 06-4: Fancy Softmax classifier 를 TensorFlow 로 구현하기

## 핵심키워드
- 다항 분류(Multinomial Classification)
- 소프트맥스(Softmax)
- 크로스 엔트로피(Cross-entropy)
- 원핫(one-hot)

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib

print(f'>>> original : tf - 2.4.0  |  np - 1.19.4')
print(f'>>> present  : tf - {tf.__version__}  |  np - {np.__version__}')

tf.random.set_seed(777)

>>> original : tf - 2.4.0  |  np - 1.19.4
>>> present  : tf - 2.4.0  |  np - 1.19.5


## Data

In [2]:
url = 'https://raw.githubusercontent.com/deeplearningzerotoall/TensorFlow/master/tf_2.x/data-04-zoo.csv'
response = urllib.request.urlopen(url)
xy = np.loadtxt(response, delimiter=',', dtype=np.float32)

x_data = xy[:, :-1]
y_data = xy[:, -1]
nb_classes = len(np.unique(y_data))  # 7
Y_one_hot = tf.one_hot(y_data, depth=nb_classes)

x_data.shape, Y_one_hot.shape

((101, 16), TensorShape([101, 7]))

## Softmax Classification

In [3]:
# Weight and bias setting
W = tf.Variable(tf.random.normal([x_data.shape[1], nb_classes]))
b = tf.Variable(tf.random.normal([nb_classes,]))

W.shape, b.shape

(TensorShape([16, 7]), TensorShape([7]))

In [4]:
# Train model
def logit_fn(X):
  return tf.matmul(X, W) + b

def hypothesis(X):
  return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
  cost_i = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logit_fn(X))
  # tf.keras.losses.categorical_crossentropy(Y, logit_fn(X), from_logits=True)
  return tf.reduce_mean(cost_i)

def grads_fn(X, Y, variables=[W, b]):
  with tf.GradientTape() as tape:
    cost = cost_fn(X, Y)
    grads = tape.gradient(cost, variables)
  return grads

def accuracy_score(X, Y):
  y_pred = tf.argmax(hypothesis(X), 1)
  y_true = tf.argmax(Y, 1)
  acc = tf.reduce_mean(tf.cast(tf.equal(y_pred, y_true), dtype=tf.float32))
  return acc

def fit(X, Y, variables=[W, b], epoch=1000, verbose=100):
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
  print('epoch |   cost   |  accuracy')

  for step in range(epoch+1):
    grads = grads_fn(X, Y, variables=variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

    if step==0 or step%verbose==0:
      cost = cost_fn(X, Y)
      acc = accuracy_score(X, Y)
      print(f'{step:5} | {cost:^8.4f} | {acc:8.4f}')
  pass


fit(x_data, Y_one_hot)

epoch |   cost   |  accuracy
    0 |  3.6350  |   0.1683
  100 |  0.5159  |   0.7921
  200 |  0.3173  |   0.9109
  300 |  0.2348  |   0.9406
  400 |  0.1884  |   0.9505
  500 |  0.1582  |   0.9604
  600 |  0.1369  |   0.9901
  700 |  0.1208  |   0.9901
  800 |  0.1084  |   1.0000
  900 |  0.0983  |   1.0000
 1000 |  0.0901  |   1.0000


In [5]:
# Prediction Check
pred = hypothesis(x_data)
print(tf.argmax(pred, 1))
print(tf.argmax(Y_one_hot, 1))

tf.Tensor(
[0 0 3 0 0 0 0 3 3 0 0 1 3 6 6 6 1 0 3 0 1 1 0 1 5 4 4 0 0 0 5 0 0 1 3 0 0
 1 3 5 5 1 5 1 0 0 6 0 0 0 0 5 4 6 0 0 1 1 1 1 3 3 2 0 0 0 0 0 0 0 0 1 6 3
 0 0 2 6 1 1 2 6 3 1 0 6 3 1 5 4 2 2 3 0 0 1 0 5 0 6 1], shape=(101,), dtype=int64)
tf.Tensor(
[0 0 3 0 0 0 0 3 3 0 0 1 3 6 6 6 1 0 3 0 1 1 0 1 5 4 4 0 0 0 5 0 0 1 3 0 0
 1 3 5 5 1 5 1 0 0 6 0 0 0 0 5 4 6 0 0 1 1 1 1 3 3 2 0 0 0 0 0 0 0 0 1 6 3
 0 0 2 6 1 1 2 6 3 1 0 6 3 1 5 4 2 2 3 0 0 1 0 5 0 6 1], shape=(101,), dtype=int64)
